In [1]:
import numpy as np
import pandas as pd
from numpy import random

import re

import random

import warnings
warnings.filterwarnings('ignore')


In [7]:
train = pd.read_csv('../model/data/train_rand.csv')
test = pd.read_csv('../model/data/test_rand.csv')

In [8]:
# trash(random) 모델 생성 작업.
# 모델 작업을 할 것이면 여기에 모델 제작하면 됨.
import torch
import torch.nn as nn
import torch.nn.functional as F
from numpy import random

# 랜덤 추천(아에 랜덤 음식점 추천)
# recall 0.001
class trash_model(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, user, rest):
        return random.randn(len(user))

model = trash_model()
model([1],[2])

array([-1.56007545])

In [9]:
# 인기도 추천(상위 3개 음식점만)
# recall 0.005
class pop_model(nn.Module):
    def __init__(self, train):
        super().__init__()
        self.rest_poplist = train['rest'].value_counts()
        
    def forward(self, user, rest):
        return np.where(np.isin(rest, self.rest_poplist.iloc[:3].index), 1 , 0)

#model = pop_model(train)

In [10]:
lst_rest = train['rest'].unique()
lst_user = train['userid'].unique()
n_rest = train['rest'].nunique()
#lst_rest = torch.LongTensor(lst_rest)

def recallk(actual, predicted, k = 3):
    """ label과 prediction 사이의 recall 평가 함수 
    Args:
        actual : 실제로 본 상품 리스트
        pred : 예측한 상품 리스트
        k : 상위 몇개의 데이터를 볼지 (ex : k=5 상위 5개의 상품만 봄)
    Returns: 
        recall_k : recall@k 
    """ 
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

def funs(user_id):
    user_ids = np.full(n_rest, user_id)
    #user_ids = torch.LongTensor(user_ids)
    output = model(user_ids, lst_rest)
    idx = np.argpartition(output, -20)[-20:]
    pred_u = lst_rest[idx]

    return pred_u
    
test = test.groupby('userid')['rest'].unique().to_frame().reset_index()
test['pred'] = test['userid'].apply(lambda x : funs(x))
test['recall'] = test.apply(lambda x : recallk(x['rest'],x['pred']), axis = 1)

In [11]:
test['recall'].mean()

0.0001061959227729323